# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.core import Environment,ScriptRunConfig
from azureml.train.estimator import Estimator
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, normal, choice
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Dataset
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice, Webservice
from train import clean_data
from sklearn.model_selection import train_test_split
import pandas as pd
from azureml.train.automl import AutoMLConfig

import os

import json
import joblib


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'automl_leukemia_experiment'

# Enviornment for model deployment
myenv = Environment(workspace=ws,name="myenv")
 
auto_exp = Experiment(ws,experiment_name)

In [3]:
# Create compute cluster

cpu_cluster_name = "cluster-leukemia"

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',min_nodes=0,
                                                            max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


### Dataset is hosted on Github

In [4]:
dataset_path = 'https://raw.githubusercontent.com/PhilippRamjoue/Leukemia_Classification/main/dataset/bone-marrow-dataset.csv'

ds = Dataset.Tabular.from_delimited_files(path=dataset_path)

In [5]:

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

# use full dataset but provide test data
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.4,random_state=42)
#automl_training_set = pd.concat([x_train,y_train],axis=1)

automl_training_set = pd.concat([x,y],axis=1)

survival_status_0: 36
survival_status_1: 85
survival_status_ongoing: 66
Shape of original frame: (121, 37)
Shape of frame with dropped nan rows: (104, 37); Loss: 14 %


## Save data in datastore

A datastore is necessary to run AutoML on a remote compute

In [6]:

# Create a project_folder if it doesn't exist
if not os.path.isdir('data'):
    os.mkdir('data')

if not os.path.exists('project_folder'):
    os.makedirs('project_folder')

automl_training_set.to_csv('data/cleaned_dataset.csv')

ds = ws.get_default_datastore()

ds.upload(src_dir='./data', target_path='leukemiadata', overwrite=True, show_progress=True)

training_data = Dataset.Tabular.from_delimited_files(path=ds.path('leukemiadata/cleaned_dataset.csv'))

Uploading an estimated of 1 files
Uploading ./data/cleaned_dataset.csv
Uploaded ./data/cleaned_dataset.csv, 1 files out of an estimated total of 1
Uploaded 1 files


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [7]:
# TODO: Put your automl settings here
automl_settings = {'experiment_timeout_minutes':30,
                    'primary_metric':'accuracy',
                    'enable_early_stopping':True,
                    'n_cross_validations':5,
                    'max_concurrent_iterations':4
                    }

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    task="classification",
    compute_target=cpu_cluster,
    training_data=training_data,
    iterations=30,
    iteration_timeout_minutes=5,
    label_column_name='survival_status',
    **automl_settings
    )

In [8]:
# TODO: Submit your experiment
auto_run = auto_exp.submit(config=automl_config,show_output=True)

Running on remote.
Running on remote compute: cluster-leukemia
Parent Run ID: AutoML_0dc345fc-190d-4af4-9971-13c998ca6441

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

***********************************

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [9]:
RunDetails(auto_run).show()


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [30]:
# Retrieve and save your best automl model.
best_run,fitted_model = auto_run.get_output(iteration=28)

In [11]:
#Example code from https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-features#scaling-and-normalization

from pprint import pprint
model = fitted_model

def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators': list(
                e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0] + ' - ')
        else:
            pprint(step[1].get_params())
            print()

print_model(model)

datatransformer
{'enable_dnn': None,
 'enable_feature_sweeping': None,
 'feature_sweeping_config': None,
 'feature_sweeping_timeout': None,
 'featurization_config': None,
 'force_text_dnn': None,
 'is_cross_validation': None,
 'is_onnx_compatible': None,
 'logger': None,
 'observer': None,
 'task': None,
 'working_dir': None}

prefittedsoftvotingclassifier
{'estimators': ['17', '27', '13', '26', '1', '2', '5', '18'],
 'weights': [0.25,
             0.08333333333333333,
             0.08333333333333333,
             0.16666666666666666,
             0.08333333333333333,
             0.16666666666666666,
             0.08333333333333333,
             0.08333333333333333]}

17 - standardscalerwrapper
{'class_name': 'StandardScaler',
 'copy': True,
 'module_name': 'sklearn.preprocessing._data',
 'with_mean': True,
 'with_std': True}

17 - randomforestclassifier
{'bootstrap': False,
 'ccp_alpha': 0.0,
 'class_weight': 'balanced',
 'criterion': 'entropy',
 'max_depth': None,
 'max_features':

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.

In [32]:
# TODO: save best AutoML model
model_name = best_run.properties['model_name']
print(model_name)
joblib.dump(fitted_model,'best_automl_model.pkl')
automl_model= auto_run.register_model(model_name=model_name)


AutoML0dc345fc128


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [41]:
# Code from https://docs.microsoft.com/en-us/python/api/overview/azure/ml/?view=azure-ml-py#model-deploy

myenv.python.conda_dependencies.add_pip_package("joblib")

# Combine scoring script & environment in Inference configuration
inference_config = InferenceConfig(entry_script="score.py",
                                   environment=myenv)

# Set deployment configuration
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1,
                                                       memory_gb = 1)

# Define the model, inference, & deployment configuration and web service name and location to deploy
service = Model.deploy(workspace = ws,
                       name = "webservice-final-3",
                       models = [automl_model],
                       inference_config = inference_config,
                       deployment_config = deployment_config)

service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running..................................................................................................................................................................................................
Failed


ERROR - Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 4dad69e0-89d6-494e-91ad-71c1c90a013a
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.\nPlease check the logs for your container instance: webservice-final-3. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs. \nYou can also try to run image 2f28b8821796430cb04560260920675a.azurecr.io/azureml/azureml_e303acaed3886f9c9130c3f485865120 locally. Please refer to https://aka.ms/debugimage#service-launch-fails for more information.",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Your container application crashed. This may be caused by errors in your scoring file's init() function.\nPlease check the logs 

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 4dad69e0-89d6-494e-91ad-71c1c90a013a
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.\nPlease check the logs for your container instance: webservice-final-3. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs. \nYou can also try to run image 2f28b8821796430cb04560260920675a.azurecr.io/azureml/azureml_e303acaed3886f9c9130c3f485865120 locally. Please refer to https://aka.ms/debugimage#service-launch-fails for more information.",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Your container application crashed. This may be caused by errors in your scoring fil

In [51]:
print(service.get_logs())

None


In [22]:
automl_training_set[:1].to_json(orient='records')


'[{"donor_age_below_35":1,"donor_CMV":0,"recipient_age_below_10":1,"recipient_gender":1,"recipient_rh":1,"recipient_CMV":0,"gender_match":0,"ABO_match":1,"HLA_mismatch":1,"risk_group":0,"stem_cell_source":1,"tx_post_relapse":0,"ANC_recovery":1,"PLT_recovery":1,"acute_GvHD_II_III_IV":1,"acute_GvHD_III_IV":0,"relapse":1,"donor_age":0.1277579149,"recipient_age":0.1436170213,"recipient_body_mass":0.1388733965,"CD34_x1e6_per_kg":0.0650991402,"CD3_x1e8_per_kg":0.0230817218,"CD3_to_CD34_ratio":0.1079678604,"time_to_ANC_recovery":0.6153846154,"time_to_PLT_recovery":0.1497975709,"time_to_acute_GvHD_III_IV":0.0,"donor_ABO_0":0,"donor_ABO_A":0,"donor_ABO_AB":0,"donor_ABO_B":1,"recipient_age_int_0_5":1,"recipient_age_int_10_20":0,"recipient_age_int_5_10":0,"recipient_ABO_0":0,"recipient_ABO_A":0,"recipient_ABO_AB":0,"recipient_ABO_B":1,"disease_ALL":1,"disease_AML":0,"disease_chronic":0,"disease_lymphoma":0,"disease_nonmalignant":0,"disease_group_malignant":1,"disease_group_nonmalignant":0,"CMV_st

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
if service.state == 'Healthy':
    x_test_json = x_test[:1].to_json(orient='records')    
    output = service.run(x_test_json)
    print(output['predictions'])



TODO: In the cell below, print the logs of the web service and delete the service

In [50]:
service.delete()

In [ ]:
# delete compute cluster
cpu_cluster.delete()
